In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<username>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['arima_LaborParticipation_pred_score',
 'CPI_RegionUrban_raw',
 'BusinessApplications_clean',
 'arima_GDP_2019_2024',
 'employment_prediction',
 'Employment_clean',
 'CPI_RegionUrban_clean',
 'population_predicted_2024',
 'employment_clean',
 'LaborForce_clean',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'Population_clean',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'msa_codes',
 'arima_unem_pred_score',
 'population_raw',
 'arima_emp_pred_score',
 'LaborForce_raw',
 'arima_cpi_pred_score',
 'UnemploymentRate_clean',
 'Employment_raw',
 'arima_2024_ROC_score_total',
 'regions_divisions',
 'arima_business_pred_score',
 'LaborParticipation_clean',
 'unemployment_raw',
 'AnnualMeanWage_clean',
 'GDP_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'arima_GDP_pred_score',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
# Select the collection with needed data
pop = db.Population_clean
df = pd.DataFrame(list(pop.find()))
print(df.dtypes)
df

_id     object
CBSA     int64
2010     int64
2011     int64
2012     int64
2013     int64
2014     int64
2015     int64
2016     int64
2017     int64
2018     int64
2019     int64
dtype: object


,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dc65f680af8db371b2438,11540,225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,603dc65f680af8db371b2439,12020,193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,603dc65f680af8db371b243a,14010,169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,603dc65f680af8db371b243b,15540,211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,603dc65f680af8db371b243c,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dc65f680af8db371b25b3,38340,131317,130547,130284,129488,128856,127858,127067,126353,125901,124944
380,603dc65f680af8db371b25b4,41700,2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960
381,603dc65f680af8db371b25b5,42100,263147,264858,266340,268816,270861,273503,274396,274856,273841,273213
382,603dc65f680af8db371b25b6,44180,437349,440432,444329,448427,451556,455593,457917,462090,466273,470300


In [4]:
df.drop(columns='_id', inplace=True)
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,11540,225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,12020,193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,14010,169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,15540,211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365
...,...,...,...,...,...,...,...,...,...,...,...
379,38340,131317,130547,130284,129488,128856,127858,127067,126353,125901,124944
380,41700,2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960
381,42100,263147,264858,266340,268816,270861,273503,274396,274856,273841,273213
382,44180,437349,440432,444329,448427,451556,455593,457917,462090,466273,470300


In [5]:
# for loop to predict 2024 values
prediction2020 = []
prediction2021 = []
prediction2022 = []
prediction2023 = []
prediction2024 = []
for i in range(0,384):
    y = df.iloc[i, 1:].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    forecast = pred.values.tolist()
    prediction2020.append(forecast[0])
    prediction2021.append(forecast[1])
    prediction2022.append(forecast[2])
    prediction2023.append(forecast[3])
    prediction2024.append(forecast[4])
df['2020'] = prediction2020
df['2021'] = prediction2021
df['2022'] = prediction2022
df['2023'] = prediction2023
df['2024'] = prediction2024

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: Convergen

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [6]:
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,11540,225948,227548,228656,229978,231627,233146,234200,235811,237011,237974,2.391677e+05,2.403438e+05,2.414988e+05,2.426330e+05,2.437468e+05
1,12020,193439,194375,195984,197590,198361,202565,205402,209106,211468,213750,2.161560e+05,2.185590e+05,2.209606e+05,2.233608e+05,2.257594e+05
2,14010,169810,170909,172721,175037,174105,173141,173070,172754,172410,171517,1.714896e+05,1.714584e+05,1.714460e+05,1.714406e+05,1.714382e+05
3,15540,211538,213235,213962,214827,215676,217152,217447,218975,219939,220411,2.212703e+05,2.221337e+05,2.229918e+05,2.238445e+05,2.246917e+05
4,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365,6.153118e+05,4.990286e+05,3.195199e+05,7.679242e+04,-2.291451e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,38340,131317,130547,130284,129488,128856,127858,127067,126353,125901,124944,1.249618e+05,1.249585e+05,1.249616e+05,1.249585e+05,1.249616e+05
380,41700,2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960,2.592820e+06,2.634078e+06,2.674757e+06,2.714865e+06,2.754410e+06
381,42100,263147,264858,266340,268816,270861,273503,274396,274856,273841,273213,2.734556e+05,2.736247e+05,2.737571e+05,2.738604e+05,2.739410e+05
382,44180,437349,440432,444329,448427,451556,455593,457917,462090,466273,470300,4.739734e+05,4.776387e+05,4.812942e+05,4.849402e+05,4.885766e+05


In [7]:
df['2020'] = df['2020'].astype('int')
df['2021'] = df['2021'].astype('int')
df['2022'] = df['2022'].astype('int')
df['2023'] = df['2023'].astype('int')
df['2024'] = df['2024'].astype('int')
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,11540,225948,227548,228656,229978,231627,233146,234200,235811,237011,237974,239167,240343,241498,242632,243746
1,12020,193439,194375,195984,197590,198361,202565,205402,209106,211468,213750,216155,218559,220960,223360,225759
2,14010,169810,170909,172721,175037,174105,173141,173070,172754,172410,171517,171489,171458,171446,171440,171438
3,15540,211538,213235,213962,214827,215676,217152,217447,218975,219939,220411,221270,222133,222991,223844,224691
4,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365,615311,499028,319519,76792,-229145


In [8]:
df.sort_values(by='2024').head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
4,19660,590506,591926,594969,600236,608604,621166,635859,647867,658186,668365,615311,499028,319519,76792,-229145
313,16180,54981,54676,54312,53726,54082,54068,54147,54532,55202,55916,55852,55798,55780,55775,55774
232,47460,58915,59444,59329,59361,59510,59932,60008,60512,60615,60760,60777,60783,60781,60783,60781
31,21420,60765,60763,61399,62370,62790,63248,62493,61600,61095,61056,60998,60968,60950,60940,60934
200,30300,61040,61409,61413,61927,62031,62108,62422,62798,62872,62990,63185,63385,63584,63784,63984


In [9]:
# create new collection in mongo
arima_pop_2019_2024 = db.arima_pop_2019_2024

In [10]:
df_dict = df.to_dict(orient='records')

In [11]:
arima_pop_2019_2024.insert_many(df_dict)